In [ ]:
#importing libraries
import cv2
import os
import time
import sys
#thresholded code to be add!
import numpy as np
# Main function to perform the splitting
def line_splitter(input_image):
    # Main list to store the output ligatures i.e. a list of numpy arrays.
    out_image_array=[]
    
    # Variable for storing the total number of lines.
    total_lines = 0
    
    # Extracting the dimensions of the image
    height,width=input_image.shape
    
    # Making a bool type filter
    filter=np.ones(shape=(1,width),dtype=bool)

    # Converting the source image to bool.
    input_image=input_image.astype(bool)
    # Some workplace variables for the following for loop.
    row=0   
    first_row=True
    line_detected=False

    # for loop for iterating through each pixel wide line of the page and separates them.
    for i in input_image:
        i=np.reshape(i,newshape=(1,width))
        res=np.bitwise_and(i,filter)
        res=np.bitwise_and.reduce(res,axis=1)

        if res==False:  #black pixel is identified
            line_detected=True
            if first_row==True:
                row_start=row
                first_row=False
            row = row + 1
            continue

        if line_detected==True:
            row_end=row
            line_detected=False
            first_row = True
            out_image=input_image[row_start:row_end,:] #cropping image 
            out_image=out_image.astype(int)  #Converting back to int.
            out_image=out_image*255          #Replacing the 1's by 255's.
            # Writing the output.
            out_image_array.append(out_image)
            total_lines=total_lines+1
        row = row + 1
   
    return out_image_array  # Returning the list

# Testing the function
# Selecting the output directory.
OUTDIR='line'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)

# Selecting the source directory
IMDIR='th'

# list to store the results.
list_of_lines=[]

# Making a counter to keep track of the images.
counter=0

# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e

# The main for loop iterating through each image.
for image in sorted(os.listdir(IMDIR),key=file_sorter):
    digitized_image = cv2.imread(os.path.join(IMDIR, image), cv2.IMREAD_GRAYSCALE)
    list_of_lines=line_splitter(digitized_image) #calling the splitter function

    for line in list_of_lines: #saving each line in output directory
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(counter)), line)
        counter = counter + 1
    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()
# Main function for performing the splitting. 
#Just takes the line image and separates the ligatures.
def splitter(input_image):
    
    # Main list to store the output ligatures i.e. a list of numpy arrays.
    out_image_array=[]
    
    # Variable for storing the total number of ligatures.
    total_ligatures=0
    
    # Flipping the image counter clockwise.
    input_image = cv2.transpose(input_image)
    input_image = cv2.flip(input_image, flipCode=0)

    # Extracting the dimensions of the image
    height, width = input_image.shape

    # Making a bool type filter
    filter = np.ones(shape=(1, width), dtype=bool)

    # Converting the source image to bool.
    input_image = input_image.astype(bool)

    # Some workplace variables for the following for loop.
    row = 0  # This variable indexs the pixels of the image row wise.
    first_row = True
    ligature_detected = False

    # This for loop iterates through each pixel wide line of the line and separates the ligatures.
    for i in input_image:
        i = np.reshape(i, newshape=(1, width))
        # Applying the operation of the filter with the row.
        res = np.bitwise_and(i, filter)
        # Reducing the result.
        res = np.bitwise_and.reduce(res, axis=1)

        if res == False:
            ligature_detected = True
            if first_row == True:
                row_start = row
                first_row = False
            row = row + 1
            continue

        if ligature_detected == True:
            row_end = row
            ligature_detected = False
            first_row = True
            out_image = input_image[row_start:row_end, :]  # Cropping the image.
            out_image = out_image.astype(int)  # Converting back to int.
            out_image = out_image * 255  # Replacing the 1's by 255's
            # Flipping the output image back in clockwise direction.
            out_image = cv2.transpose(out_image)
            out_image = cv2.flip(out_image, flipCode=+1)
            # Writing the output to the array
            out_image_array.append(out_image)
            total_ligatures=total_ligatures+1

        row = row + 1
    # print("Ligatures separated successfully. Total ligatures {}.".format(total_ligatures))
    # Returning the list..
    return out_image_array

# Executing the function:*****************************************************

# Selecting the output directory.
OUTDIR='lig'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
# Selecting the source directory
IMDIR='line'

# The function returns a list of ligatures. Making that list:
ligs=[]

# This increments for each ligature image separated
counter=0

# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e

# Main for loop iterating through the images.
for image in sorted(os.listdir(IMDIR), key=file_sorter):
   
    digitized_image = cv2.imread(os.path.join(IMDIR, image), cv2.IMREAD_GRAYSCALE)  # Reading the image
    ligs=splitter(input_image=digitized_image)     #Calling the function of Ligature spliiter

    for l in ligs:
        # Writing the output.
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(str(counter))), l)
        counter=counter+1

    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()
    
#Function for stretching ligatures
def stretcher(input_image,degree_of_stretch):
    # Getting the dimensions
    height_original, width_original = input_image.shape
    out_image_array=[]
    
    # Making the height of the image divisible by degree_of_stretch
    while(height_original%degree_of_stretch!=0):
        height_original=height_original+1
        input_image=np.insert(input_image,0,255,0)
    # degree_of_stretch:
    # How many pixels to skip for the shifting operation.
    # OR we can say that it is the degree of stretch.
    final_width = width_original + int(height_original / degree_of_stretch)
    img_modified = np.ones((height_original, final_width))
    img_modified=img_modified*255
    counter = 0
    inserts = 0    #paddings
    for row in img_modified:
        if counter % degree_of_stretch == 0:
            inserts = inserts + 1
        row[0 + inserts: width_original + inserts] = input_image[counter] #  row[2:22] replace by input_image[1]
        counter = counter + 1
    out_image_array.append(img_modified)
    #return img_modified
    return out_image_array

# Selecting the input directory 
IMDIR='lig'
# Selecting the output directory for small ligatures.
OUTDIR='str'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
    
#Mkaing a list for saving streteched ligatures    
list_of_stretched=[]

# Making a counter to keep track of the images.
counter=0

# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e

# Main for loop 
for image in os.listdir(IMDIR):
    # Reading the input image.
    img_original = cv2.imread(os.path.join(IMDIR, image), cv2.IMREAD_GRAYSCALE)
    list_of_stretched=stretcher(img_original,2)
    
    for ligatures in list_of_stretched:
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(counter)), ligatures)
        counter = counter + 1
    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()
    
#importing libraries
import cv2
import os
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
import pdb
import scipy.io as sio
from PIL import Image
import matplotlib.patches as mpatches
from skimage import measure
from scipy import ndimage
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

#Selecting input directory
IMDIR='str'
#selecting output directory to save dilated ligature images
OUTDIR ='dots'
#Selecting output directory to save images whose first component is non-diacritic
OUTDIRo='nodots'
#Setting the counter value
counter=0
#List for saving dilating images
dilated_image=[]
#Main For loop
for image in os.listdir(IMDIR):
    input_image =cv2.imread(os.path.join(IMDIR,image),cv2.IMREAD_UNCHANGED)
    #Setting some variable values for connected component method
    connectivity=8
    blur_radius =0.6
    threshold = 100
    #Rotating the input image
    transposed_image = cv2.transpose(input_image)
    rotated_image = cv2.flip(transposed_image, flipCode =0)
    input_image = rotated_image
    imgf = ndimage.gaussian_filter(input_image, blur_radius)
    threshold = 100
    input_image = (imgf < threshold).astype(np.uint8)
    #Applying connected component analysis method
    nlabels,labels, stats, centroids = cv2.connectedComponentsWithStats(input_image)
# plt.imshow(labels)
   # plt.show()
    stats=stats.astype(np.uint16)
    LABEL_ARRAY = np.arange(1, nlabels)
    n = 0
    out_image_array=[]
    for region in regionprops(labels): #iterating each label
        n+=1
        if n <=1 and region.extent > 0.51: #separating liagture if criteria matches
            width = stats[n, cv2.CC_STAT_WIDTH]
            height = stats[n, cv2.CC_STAT_HEIGHT]
            x = stats[n, cv2.CC_STAT_LEFT]
            y = stats[n, cv2.CC_STAT_TOP]
            roi = input_image[y:y+height+10,x:x+width] #selecting the values for dilation
            kernel=cv2.getStructuringElement(cv2.MORPH_CROSS,(4,4)) #dialting structure
            roi[:]=cv2.dilate(roi, kernel, iterations=1) #dilating selected region
            out_image = cv2.transpose(input_image)
            out_image = cv2.flip(out_image, flipCode=+1) #flipping back image 
            out_image_array.append(out_image)#saving in list
            plt.imsave(os.path.join(OUTDIR, "{}.png".format(str(counter))),out_image) #saving into one folder
            
        elif n <=1 and region.extent < 0.52:
            in_image = cv2.transpose(input_image)
            in_image = cv2.flip(in_image, flipCode=+1)
            plt.imsave(os.path.join(OUTDIRo, "{}.png".format(counter)), in_image) #passing into other folder
    counter=counter+1
    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()
    
# The input images to this file must be line separated and binary.
import cv2
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import sys

# Main function for performing the splitting. Just takes the line image and separates the ligatures.
def splitter(input_image):
    # Main list to store the output ligatures i.e. a list of numpy arrays.
    out_image_array=[]
    
    # Variable for storing the total number of ligatures.
    total_ligatures=0
    
    # Flipping the image counter clockwise.
    input_image = cv2.transpose(input_image)
    input_image = cv2.flip(input_image, flipCode=0)
    
    # Extracting the dimensions of the image
    height, width = input_image.shape

    # Making a filter
    filter = np.full((1, width),30)

    # Some workplace variables for the following for loop.
    row = 0  # This variable indexs the pixels of the image row wise.
    first_row = True
    ligature_detected = False

    # This for loop iterates through each pixel wide line of the line and separates the ligatures.
    for i in input_image:
        i = np.reshape(i, newshape=(1, width))
        # Applying the operation of the filter with the row.
        res = np.subtract(i, filter)
        # Reducing the result.
        res = np.subtract.reduce(res, axis=1)

        if res !=0:
            ligature_detected = True
            if first_row == True:
                row_start = row
                first_row = False
            row = row + 1
            continue

        if ligature_detected == True:
            row_end = row
            ligature_detected = False
            first_row = True
            out_image = input_image[row_start:row_end, :]  # Cropping the image.
            out_image = cv2.transpose(out_image)
            out_image = cv2.flip(out_image, flipCode=+1)
            # Writing the output to the array
            out_image_array.append(out_image)
            total_ligatures=total_ligatures+1

        row = row + 1
    print("Ligatures separated successfully. Total ligatures {}.".format(total_ligatures))
    # Returning the list
    return out_image_array

# Executing the function:

# Selecting the output directory.
OUTDIR='dots-splitted'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
# Selecting the source directory
IMDIR='dots'
# The function returns a list of ligatures. Making that list:
ligs=[]
# This increments for each ligature image separated
counter=0
# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e
# Main for loop iterating through the images.
for image in sorted(os.listdir(IMDIR), key=file_sorter):
    # Reading the image.
    digitized_image = cv2.imread(os.path.join(IMDIR, image),0)
    ligs=splitter(digitized_image)

    for l in ligs:
        # Writing the output.
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(str(counter))), l)
        counter=counter+1

    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()
    
#Fuunction for destretching images
def deStretcher(input_image,degree_of_stretch):
    
    # Getting the dimensions
    height_original, width_original = input_image.shape
    # Making a list to save output
    out_image_array=[]
    # Making the height of the image divisible by degree_of_stretch
    while (height_original % degree_of_stretch != 0):
        height_original = height_original + 1
        original_image = np.insert(original_image, 0,0, 0) #padding pixel values
        
    final_width = width_original + int(height_original/ degree_of_stretch)
    img_modified = np.ones((height_original, final_width))
    img_modified = img_modified *30 #modifying pixel values
    counter = 0
    inserts = 0
    for row in img_modified:
        if counter % degree_of_stretch == 0:
            inserts = inserts + 1
        row[final_width - width_original - inserts:final_width - inserts] = input_image[counter]
        counter = counter + 1
    out_image_array.append(img_modified)
    #return img_modified
    return out_image_array

# Testing the function:

# Selecting the source directory
IMDIR='dots-splitted'

# Selecting the output directory for small ligatures.
OUTDIR='dots-destr'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
    
list_of_destretched=[]

# Making a counter to keep track of the images.
counter=0

# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e

# Main for loop to rotate the images:
for image in os.listdir(IMDIR):
    # Reading the input image.
    img_original = cv2.imread(os.path.join(IMDIR, image), cv2.IMREAD_GRAYSCALE)
    list_of_destretched=deStretcher(img_original,2)
    
    for ligatures in list_of_destretched:
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(counter)), ligatures)
        counter = counter + 1
    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()

Images Left 191349/225709.

In [1]:
import cv2
import os
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
import pdb
import scipy.io as sio
from PIL import Image
import matplotlib.patches as mpatches
from skimage import measure
from scipy import ndimage
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
def stretcher(input_image,degree_of_stretch):
    # Getting the dimensions
    height_original, width_original = input_image.shape
    out_image_array=[]
    
    # Making the height of the image divisible by degree_of_stretch
    while(height_original%degree_of_stretch!=0):
        height_original=height_original+1
        input_image=np.insert(input_image,0,255,0)
    # degree_of_stretch:
    # How many pixels to skip for the shifting operation.
    # OR we can say that it is the degree of stretch.
    final_width = width_original + int(height_original / degree_of_stretch)
    img_modified = np.ones((height_original, final_width))
    img_modified=img_modified*255
    counter = 0
    inserts = 0    #paddings
    for row in img_modified:
        if counter % degree_of_stretch == 0:
            inserts = inserts + 1
        row[0 + inserts: width_original + inserts] = input_image[counter] #  row[2:22] replace by input_image[1]
        counter = counter + 1
    out_image_array.append(img_modified)
    #return img_modified
    return out_image_array

# Selecting the input directory 
IMDIR='lig'
# Selecting the output directory for small ligatures.
OUTDIR='str'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
    
#Mkaing a list for saving streteched ligatures    
list_of_stretched=[]

# Making a counter to keep track of the images.
counter=0

# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e

# Main for loop 
for image in os.listdir(IMDIR):
    # Reading the input image.
    img_original = cv2.imread(os.path.join(IMDIR, image), cv2.IMREAD_GRAYSCALE)
    list_of_stretched=stretcher(img_original,2)
    
    for ligatures in list_of_stretched:
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(counter)), ligatures)
        counter = counter + 1
    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()
    

Images Left 163786/221924.

KeyboardInterrupt: 

In [ ]:
# The input images to this file must be line separated and binary.
import cv2
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import sys

# Main function for performing the splitting. Just takes the line image and separates the ligatures.
def splitter(input_image):
    # Main list to store the output ligatures i.e. a list of numpy arrays.
    out_image_array=[]
    
    # Variable for storing the total number of ligatures.
    total_ligatures=0
    
    # Flipping the image counter clockwise.
    input_image = cv2.transpose(input_image)
    input_image = cv2.flip(input_image, flipCode=0)
    
    # Extracting the dimensions of the image
    height, width = input_image.shape

    # Making a filter
    filter = np.full((1, width),30)

    # Some workplace variables for the following for loop.
    row = 0  # This variable indexs the pixels of the image row wise.
    first_row = True
    ligature_detected = False

    # This for loop iterates through each pixel wide line of the line and separates the ligatures.
    for i in input_image:
        i = np.reshape(i, newshape=(1, width))
        # Applying the operation of the filter with the row.
        res = np.subtract(i, filter)
        # Reducing the result.
        res = np.subtract.reduce(res, axis=1)

        if res !=0:
            ligature_detected = True
            if first_row == True:
                row_start = row
                first_row = False
            row = row + 1
            continue

        if ligature_detected == True:
            row_end = row
            ligature_detected = False
            first_row = True
            out_image = input_image[row_start:row_end, :]  # Cropping the image.
            out_image = cv2.transpose(out_image)
            out_image = cv2.flip(out_image, flipCode=+1)
            # Writing the output to the array
            out_image_array.append(out_image)
            total_ligatures=total_ligatures+1

        row = row + 1
    print("Ligatures separated successfully. Total ligatures {}.".format(total_ligatures))
    # Returning the list
    return out_image_array

# Executing the function:

# Selecting the output directory.
OUTDIR='nodots-splitted'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)
# Selecting the source directory
IMDIR='nodots'
# The function returns a list of ligatures. Making that list:
ligs=[]
# This increments for each ligature image separated
counter=0
# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e
# Main for loop iterating through the images.
for image in sorted(os.listdir(IMDIR), key=file_sorter):
    # Reading the image.
    digitized_image = cv2.imread(os.path.join(IMDIR, image),0)
    ligs=splitter(digitized_image)

    for l in ligs:
        # Writing the output.
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(str(counter))), l)
        counter=counter+1

    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()
    

Ligatures separated successfully. Total ligatures 1.
Images Left 0/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 2/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 3/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 4/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 6/41433.Ligatures separated successfully. Total ligatures 3.
Images Left 7/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 8/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 11/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 13/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 14/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 17/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 18/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 19/41433.Ligatures separated successfully. Total ligat

Images Left 169/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 170/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 171/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 174/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 175/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 176/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 177/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 178/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 179/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 180/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 181/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 182/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 183/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 184/41433.Lig

Images Left 493/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 494/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 495/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 496/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 497/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 498/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 500/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 505/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 506/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 507/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 509/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 510/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 511/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 513/41433.Lig

Images Left 902/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 904/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 905/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 906/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 910/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 912/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 913/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 914/41433.Ligatures separated successfully. Total ligatures 4.
Images Left 915/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 916/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 917/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 918/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 921/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 925/41433.Lig

Images Left 1109/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 1110/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 1113/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 1117/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 1118/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 1119/41433.Ligatures separated successfully. Total ligatures 7.
Images Left 1120/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 1124/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 1125/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 1131/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 1132/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 1133/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 1134/41433.Ligatures separated successfully. Total ligatures 3.
Images Left 

...

Images Left 34759/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 34761/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 34762/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34763/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34764/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34765/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34766/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 34767/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34770/41433.Ligatures separated successfully. Total ligatures 4.
Images Left 34771/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34772/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 34774/41433.Ligatures separated successfully. Total ligatures 4.
Images Left 34775/41433.Ligatures separated successfully. Total ligatures 4.

Images Left 34931/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34932/41433.Ligatures separated successfully. Total ligatures 4.
Images Left 34933/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34936/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34937/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34938/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34939/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34941/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34942/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34943/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 34945/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34946/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 34947/41433.Ligatures separated successfully. Total ligatures 2.

Images Left 35253/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35254/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35255/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35256/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35257/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 35258/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35261/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35263/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 35264/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35265/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35266/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35267/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 35269/41433.Ligatures separated successfully. Total ligatures 1.

Images Left 35411/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35412/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35416/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35417/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 35418/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35419/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35420/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35421/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35422/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35423/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35424/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35428/41433.Ligatures separated successfully. Total ligatures 2.
Images Left 35429/41433.Ligatures separated successfully. Total ligatures 2.

Images Left 35560/41433.Ligatures separated successfully. Total ligatures 1.
Images Left 35561/41433.Ligatures separated successfully. Total ligatures 3.
